In [ ]:
from googletrans import Translator

In [ ]:
!pip install httpx==0.13.3

In [ ]:
translator = Translator()
translator.translate('안녕하세요.',dest='ja')

In [ ]:
!pip install -U deep-translator


In [ ]:
GoogleTranslator(source='auto', target='de').translate("keep it up, you are awesome") 
'weiter so, du bist toll'

In [ ]:
from pptx import Presentation
from deep_translator import GoogleTranslator
from docx import Document

# pip install -U deep-translator
# pip install python-pptx

# Load presentationj
ppt = Presentation("./Translator.pptx")
translator = Translator()

for slide in ppt.slides:
    for shape in slide.shapes:
        if hasattr(shape, "text"):
            original_text = shape.text
            translated_text = GoogleTranslator(source='auto', target = 'vi').translate(original_text)
            shape.text = translated_text

# Save the translated presentation
ppt.save("translated_presentation.pptx")

In [10]:
import fitz  # PyMuPDF
from deep_translator import GoogleTranslator
import unicodedata
import os


# Define the path to the fonts directory
fonts_dir = "./fonts"  # Adjust this path as needed
font_files = ['OpenSans.ttf']

# Open the PDF
pdf_document = fitz.open('./test_document.pdf')
no_pages = len(pdf_document)

# Load fonts from files
loaded_fonts = []
for font_file in font_files:
    font_path = os.path.join(fonts_dir, font_file)
    if os.path.exists(font_path):
        try:
            font_name = font_file.split('.')[0]
            loaded_fonts.append((font_name, font_path))
            print(f"Font file loaded: {font_file}")
        except Exception as e:
            print(f"Failed to load font {font_file}: {str(e)}")

if not loaded_fonts:
    print("No fonts could be loaded. Using built-in font as fallback.")
    loaded_fonts = [("helv", None)]  # None will use a built-in font



# Create a new PDF for the translated content
translated_pdf = fitz.open()

# Process each page
for page_num in range(1):
    page = pdf_document[page_num]
    new_page = translated_pdf.new_page(width=page.rect.width, height=page.rect.height)
    
    # Get all text blocks on the page
    blocks = page.get_text("dict")["blocks"]
    
    for block in blocks:
        if block["type"] == 0:  # Text block
            for line in block["lines"]:
                for span in line["spans"]:
                    original_text = span["text"]
                    font_size = span["size"]
                    
                    # Translate the text
                    translated_text = GoogleTranslator(source='auto', target='vi').translate(original_text)
                    
                    # Try to insert text with loaded fonts
                    for font_name, font_path in loaded_fonts:
                        try:
                            if font_path:
                                new_page.insert_font(fontname=font_name, fontfile=font_path)
                            new_page.insert_text(
                                (span["origin"][0], span["origin"][1]),
                                translated_text,
                                fontsize=font_size,
                                fontname=font_name,
                                color=(0, 0, 0)
                            )
                            break  # If successful, break the loop
                        except Exception as e:
                            if (font_name, font_path) == loaded_fonts[-1]:  # If it's the last font in the list
                                print(f"Failed to insert text: {translated_text}")
                                print(f"Error: {str(e)}")
        elif block["type"] == 1:  # Image block
            try:
                # Copy images from the original PDF to the new PDF
                new_page.insert_image(block["bbox"], stream=page.get_images()[block["number"]][1])
            except Exception as e:
                print(f"Failed to insert image. Error: {str(e)}")

# Save the modified PDF
translated_pdf.save('translated_output.pdf')
translated_pdf.close()
pdf_document.close()
print("Translation complete. Saved as 'translated_output.pdf'.")

Font file loaded: OpenSans.ttf
Translation complete. Saved as 'translated_output.pdf'.


## Translate PDF

In [ ]:
!pip install PyMuPDF

In [ ]:
import fitz  # PyMuPDF
from deep_translator import GoogleTranslator

# Open the PDF
pdf_document = fitz.open('./test_document.pdf')
no_pages = len(pdf_document)

# Process each page
for page_num in range(no_pages):
    page = pdf_document[page_num]
    
    # Get all text blocks on the page
    blocks = page.get_text("blocks")
    
    # Clear the page content
    page.clean_contents()
    
    for block in blocks:
        # Extract text and its position
        original_text = block[4]
        x0, y0, x1, y1 = block[:4]
        
        # Translate the text
        translated_text = GoogleTranslator(source='auto', target='vi').translate(original_text)
        
        # Insert the translated text at the original position
        page.insert_text((x0, y0), translated_text, fontsize=11, color=(0, 0, 0))

# Save the modified PDF
pdf_document.save('translated_example.pdf')
pdf_document.close()
print("Translation complete. Saved as 'translated_example.pdf'.")